# Import

In [1]:
import mmdet
import mmcv
import mmengine
from mmengine import Config
from mmengine.runner import set_random_seed
#from mmdet.datasets import build_dataset
#from mmdet.models import build_detector
from mmengine.runner import Runner
from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings('ignore') #Ignore "future" warnings and Data-Frame-Slicing warnings.

from sklearn.model_selection import KFold, StratifiedKFold
from pytorch_lightning import seed_everything

# Config

In [3]:
class CFG:
    class general:
        #project_name = "HuBMAP2023"
        input_path = "../data/input"
        output_path = "../data/output"
        save_name = "resnet_all_flip"
        seed = 0
        cv = True
        #wandb_desabled = True
        n_splits = 5
        fold = [0] # list (0-idx start) or null. Set one element list, hold-out mode.

# Main

### Read data

In [4]:
tile_meta = pd.read_csv(f"{CFG.general.input_path}/tile_meta.csv")
polygons = pd.read_json(f"{CFG.general.input_path}/polygons.jsonl", lines=True)
train = pd.merge(polygons, tile_meta, on="id", how="left")
print(len(train))

1633


In [5]:
with open(f"{CFG.general.input_path}/coco_annotations_all.json") as f:
    coco_json = json.load(f)

In [6]:
coco_info = pd.DataFrame(coco_json["info"])
coco_licenses = pd.DataFrame(coco_json["licenses"])
coco_categories = pd.DataFrame(coco_json["categories"])
coco_images = pd.DataFrame(coco_json["images"])
coco_annotations = pd.DataFrame(coco_json["annotations"])

In [7]:
coco_categories

,id,name
0,1,glomerulus
1,2,blood_vessel
2,3,unsure


In [8]:
# bloodだけにするかどうか
coco_annotations = coco_annotations[coco_annotations["category_id"]==2].reset_index(drop=True)

# unsureを除くかどうか
#coco_annotations = coco_annotations[coco_annotations["category_id"]!=3].reset_index(drop=True)

# glomerulusを除くかどうか
#coco_annotations = coco_annotations[coco_annotations["category_id"]!=1].reset_index(drop=True)      

# unsureをbloodにするかどうか
#coco_annotations.loc[coco_annotations["category_id"]==3, "category_id"] = 2

coco_annotations["id"] = coco_annotations.index + 1

coco_json["categories"] = [coco_categories.iloc[1].to_dict()]
#coco_json["categories"] = coco_categories.iloc[:2].to_dict(orient="records")

coco_json["images"] = coco_images.to_dict(orient="records")
coco_json["annotations"] = coco_annotations.to_dict(orient="records")

output_file_path = f"{CFG.general.input_path}/coco_annotations_train_all.json"
with open(output_file_path, "w", encoding="utf-8") as output_file:
    json.dump(coco_json, output_file, ensure_ascii=True, indent=4)

print(f"annos: {len(coco_annotations)}, images: {len(coco_images)}")

annos: 16054, images: 1633


In [9]:
backbone = "resnet"
if backbone == "convnext":
    cfg = Config.fromfile("../mmdetection/configs/convnext/cascade-mask-rcnn_convnext-t-p4-w7_fpn_4conv1fc-giou_amp-ms-crop-3x_coco.py")
    #cfg.work_dir = ""
    #Runner.from_cfg(cfg)
    
    # cfg dataset
    cfg.metainfo = {
        #"classes": ("glomerulus", "blood_vessel", )
        "classes": ("blood_vessel", )
    }
    cfg.train_pipeline = [
            dict(type="LoadImageFromFile", backend_args=None),
            dict(type="LoadAnnotations", with_bbox=True, with_mask=True, poly2mask=True),
            dict(
                type="RandomChoiceResize",
                scales=[(640, 640), (768, 768), (896, 896), (1024, 1024), (1152, 1152), (1280, 1280), (1408, 1408), (1536, 1536)],
                keep_ratio=True),
            dict(
                type="RandomFlip",
                prob=0.0),
            dict(type="PackDetInputs")
        ]
    cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
    cfg.data_root = f"{CFG.general.input_path}"
    cfg.train_dataloader.dataset.ann_file = f"coco_annotations_train_all.json"
    cfg.train_dataloader.dataset.data_root = cfg.data_root
    cfg.train_dataloader.dataset.data_prefix.img = "train"
    cfg.train_dataloader.dataset.metainfo = cfg.metainfo
    cfg.val_dataloader = None
    cfg.val_evaluator = None
    cfg.val_cfg = None
    cfg.test_dataloader = None
    cfg.test_evaluator = None 
    cfg.test_cfg = None

    # classes
    cfg.model.roi_head.bbox_head[0].num_classes = 1
    cfg.model.roi_head.bbox_head[1].num_classes = 1
    cfg.model.roi_head.bbox_head[2].num_classes = 1
    cfg.model.roi_head.mask_head.num_classes = 1

    # pretrained weight
    cfg.load_from = "../data/pretrained_weight/cascade_mask_rcnn_convnext-t_p4_w7_fpn_giou_4conv1f_fp16_ms-crop_3x_coco_20220509_204200-8f07c40b.pth"
    
    # Set up working dir to save files and logs
    cfg.work_dir = f"{CFG.general.output_path}/{CFG.general.save_name}"

    # training configs, learning rate, optimizer
    max_iter = 5000
    cfg.train_cfg = dict(type="IterBasedTrainLoop", max_iters=max_iter, val_interval=500)
    cfg.train_dataloader.sampler = dict(type="InfiniteSampler", shuffle=True)
    cfg.log_processor = dict(by_epoch=False)
    cfg.optim_wrapper.type = "AmpOptimWrapper"
    cfg.param_scheduler = [
            dict(
                type="LinearLR", start_factor=1.0e-3, by_epoch=False, begin=0, end=int(max_iter*0.1)),
            dict(
                type="CosineAnnealingLR",
                T_max=max_iter-int(max_iter*0.1),
                by_epoch=False,
                begin=int(max_iter*0.1),
                end=max_iter)
        ]

    # logger
    cfg.default_hooks.checkpoint.interval = 0
    cfg.default_hooks.logger.interval = 100

    # Set seed thus the results are more reproducible
    set_random_seed(CFG.general.seed, deterministic=False)
    
elif backbone == "resnet":
    cfg = Config.fromfile("../mmdetection/configs/cascade_rcnn/cascade-mask-rcnn_r50_fpn_ms-3x_coco.py")
    #cfg = Config.fromfile("../mmdetection/configs/cascade_rcnn/cascade-mask-rcnn_x101-64x4d_fpn_ms-3x_coco.py")
    
    # cfg dataset
    cfg.metainfo = {
        "classes": ("blood_vessel", )
    }
    cfg.train_pipeline = [
            dict(type="LoadImageFromFile", backend_args=None),
            dict(type="LoadAnnotations", with_bbox=True, with_mask=True, poly2mask=True),
            dict(
                type="RandomChoiceResize",
                scales=[(640, 640), (768, 768), (896, 896), (1024, 1024), (1152, 1152), (1280, 1280), (1408, 1408), (1536, 1536)],
                keep_ratio=True),
            #dict(type="RandomFlip", prob=0.0),
            dict(type="RandomFlip", direction=["horizontal", "vertical"], prob=[0.5, 0.5]),
            dict(type="PackDetInputs")
        ]
    cfg.train_dataloader.dataset.dataset.pipeline = cfg.train_pipeline
    cfg.test_pipeline = [
            dict(type="LoadImageFromFile", backend_args=None),
            dict(type="Resize", scale=(1024, 1024), keep_ratio=True),
            dict(type="LoadAnnotations", with_bbox=True, with_mask=True),
            dict(
                type="PackDetInputs",
                meta_keys=("img_id", "img_path", "ori_shape", "img_shape", "scale_factor"))
        ]
    cfg.data_root = f"{CFG.general.input_path}"
    cfg.train_dataloader.dataset.dataset.ann_file = f"coco_annotations_train_all.json"
    cfg.train_dataloader.dataset.dataset.data_root = cfg.data_root
    cfg.train_dataloader.dataset.dataset.data_prefix.img = "train"
    cfg.train_dataloader.dataset.dataset.metainfo = cfg.metainfo
    cfg.val_dataloader = None
    cfg.val_evaluator = None
    cfg.val_cfg = None
    cfg.test_dataloader = None
    cfg.test_evaluator = None 
    cfg.test_cfg = None

    # classes
    cfg.model.roi_head.bbox_head[0].num_classes = 1
    cfg.model.roi_head.bbox_head[1].num_classes = 1
    cfg.model.roi_head.bbox_head[2].num_classes = 1
    cfg.model.roi_head.mask_head.num_classes = 1

    # pretrained weight
    cfg.load_from = "../data/pretrained_weight/cascade_mask_rcnn_r50_fpn_mstrain_3x_coco_20210628_164719-5bdc3824.pth"
    #cfg.load_from = "../data/pretrained_weight/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco_20210719_210311-d3e64ba0.pth"

    # Set up working dir to save files and logs
    cfg.work_dir = f"{CFG.general.output_path}/{CFG.general.save_name}"

    # training configs, learning rate, optimizer
    max_iter = 5000
    cfg.train_cfg = dict(type="IterBasedTrainLoop", max_iters=max_iter, val_interval=500)
    cfg.train_dataloader.sampler = dict(type="InfiniteSampler", shuffle=True)
    cfg.log_processor = dict(by_epoch=False)
    cfg.optim_wrapper.type = "AmpOptimWrapper"
    cfg.optim_wrapper.optimizer.type = "SGD"
    cfg.optim_wrapper.optimizer.lr = 1.0e-2
    cfg.optim_wrapper.optimizer.weight_decay = 1.0e-5
    cfg.param_scheduler = [
            dict(
                type="LinearLR", start_factor=1.0e-3, by_epoch=False, begin=0, end=int(max_iter*0.1)),
            dict(
                type="CosineAnnealingLR",
                T_max=max_iter-int(max_iter*0.1),
                by_epoch=False,
                begin=int(max_iter*0.1),
                end=max_iter)
        ]

    # logger
    cfg.default_hooks.checkpoint.interval = 0
    cfg.default_hooks.logger.interval = 100

    # Set seed thus the results are more reproducible
    set_random_seed(CFG.general.seed, deterministic=False)

elif backbone == "mask2former":
    cfg = Config.fromfile("../mmdetection/configs/mask2former/mask2former_swin-t-p4-w7-224_8xb2-lsj-50e_coco.py")
    #cfg.work_dir = "../../"
    #Runner.from_cfg(cfg)
    
    # cfg dataset
    cfg.metainfo = {
        "classes": ("blood_vessel", )
    }
    cfg.train_pipeline[2] = dict(type="RandomFlip", prob=0.0)
    cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
    cfg.test_pipeline[1] = dict(type="Resize", scale=(1024, 1024), keep_ratio=True)
    cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
    cfg.data_root = f"{CFG.general.input_path}"
    cfg.train_dataloader.dataset.ann_file = f"coco_annotations_train_fold{fold_n}.json"
    cfg.train_dataloader.dataset.data_root = cfg.data_root
    cfg.train_dataloader.dataset.data_prefix.img = "train"
    cfg.train_dataloader.dataset.metainfo = cfg.metainfo
    cfg.val_dataloader.dataset.ann_file = f"coco_annotations_valid_fold{fold_n}.json"
    cfg.val_dataloader.dataset.data_root = cfg.data_root
    cfg.val_dataloader.dataset.data_prefix.img = "train"
    cfg.val_dataloader.dataset.metainfo = cfg.metainfo
    cfg.test_dataloader = cfg.val_dataloader
    cfg.val_evaluator.ann_file = f"{CFG.general.input_path}/coco_annotations_valid_fold{fold_n}.json"
    cfg.val_evaluator.metric = "segm"
    cfg.test_evaluator = cfg.val_evaluator

    # classes
    cfg.num_things_classes = 1
    cfg.num_classes = 1
    cfg.model.panoptic_head.num_things_classes = cfg.num_things_classes
    cfg.model.panoptic_head.num_stuff_classes = cfg.num_stuff_classes
    cfg.model.panoptic_fusion_head.num_things_classes = cfg.num_things_classes
    cfg.model.panoptic_fusion_head.num_stuff_classes = cfg.num_stuff_classes
    cfg.model.panoptic_head.loss_cls["class_weight"] = [1.0] * cfg.num_classes + [0.1]

    # pretrained weight
    cfg.load_from = "../data/pretrained_weight/mask2former_swin-t-p4-w7-224_8xb2-lsj-50e_coco_20220508_091649-01b0f990.pth"

    # Set up working dir to save files and logs
    cfg.work_dir = f"{CFG.general.output_path}/{CFG.general.save_name}"

    # training configs, learning rate, optimizer
    max_iter = 10000
    cfg.train_cfg = dict(type="IterBasedTrainLoop", max_iters=max_iter, val_interval=500)
    cfg.train_dataloader.sampler = dict(type="InfiniteSampler", shuffle=True)
    cfg.log_processor = dict(by_epoch=False)
    cfg.optim_wrapper.type = "AmpOptimWrapper"
    #cfg.optim_wrapper.optimizer = dict(type="SGD", lr=0.01, momentum=0.9, weight_decay=1e-05)
    cfg.param_scheduler = [
            dict(
                type="LinearLR", start_factor=1.0e-3, by_epoch=False, begin=0, end=int(max_iter*0.1)),
            dict(
                type="CosineAnnealingLR",
                T_max=max_iter-int(max_iter*0.1),
                by_epoch=False,
                begin=int(max_iter*0.1),
                end=max_iter)
        ]

    # logger
    cfg.default_hooks.checkpoint.interval = 0
    cfg.default_hooks.logger.interval = 100

    # Set seed thus the results are more reproducible
    set_random_seed(CFG.general.seed, deterministic=False)

In [10]:
runner = Runner.from_cfg(cfg)

07/24 11:42:07 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.16 (main, Dec  7 2022, 01:11:51) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 209652396
    GPU 0: NVIDIA RTX A6000
    CUDA_HOME: /usr/local/cuda-11.6
    NVCC: Cuda compilation tools, release 11.6, V11.6.124
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=co

In [11]:
seed_everything(CFG.general.seed, workers=True)
runner.train()

Global seed set to 0


loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
07/24 11:42:13 - mmengine - INFO - load model from: torchvision://resnet50
07/24 11:42:13 - mmengine - INFO - Loads checkpoint by torchvision backend from path: torchvision://resnet50


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


07/24 11:42:18 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Loads checkpoint by local backend from path: ../data/pretrained_weight/cascade_mask_rcnn_r50_fpn_mstrain_3x_coco_20210628_164719-5bdc3824.pth
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torc

CascadeRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tr

# Val

In [12]:
runs = []
for size in [640, 768, 896, 1024, 1152, 1280, 1408, 1536]:
    print(f"[{size}]")
    cfg.load_from = f"{CFG.general.output_path}/{CFG.general.save_name}/epoch_1.pth"
    #cfg.load_from = f"{CFG.general.output_path}/{CFG.general.save_name}/iter_10000.pth"
    cfg.test_pipeline = [
            dict(type="LoadImageFromFile", backend_args=None),
            dict(type="Resize", scale=(size, size), keep_ratio=True),
            dict(type="LoadAnnotations", with_bbox=True, with_mask=True),
            dict(
                type="PackDetInputs",
                meta_keys=("img_id", "img_path", "ori_shape", "img_shape", "scale_factor"))
        ]
    cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
    runner = Runner.from_cfg(cfg)
    runs.append(runner)

[640]


AttributeError: 'NoneType' object has no attribute 'dataset'

In [ ]:
for run_, size in zip(runs, [640, 768, 896, 1024, 1152, 1280, 1408, 1536]):
    print(f"[{size}]")
    run_.val()

In [ ]:
# 640 -> 406, 768 -> 422, 896 -> 428, 1024 -> 440, 1152 -> 433, 1280 -> 439, 1408 -> 431, 1536 -> 420

# resnext
# 640 -> 410, 768 -> 435, 896 -> 441, 1024 -> 440, 1152 -> 441, 1280 -> 443, 1408 -> 432, 1536 -> 429

# convnext tiny
# 640 -> 435, 768 -> 449, 896 -> 454, 1024 -> 455, 1152 -> 456, 1280 -> 453, 1408 -> 450, 1536 -> 443

In [ ]:
img = mmcv.imread(f"{CFG.general.input_path}/train/0006ff2aa7cd.tif", channel_order="rgb")
checkpoint_file = f"{CFG.general.output_path}/{CFG.general.save_name}/epoch_1.pth"
model = init_detector(cfg, checkpoint_file)
new_result = inference_detector(model, img)
print(new_result)

In [ ]:
# get built visualizer
visualizer_now = Visualizer.get_current_instance()
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_detector
visualizer_now.dataset_meta = model.dataset_meta
# show the results
visualizer_now.add_datasample(
    "new_result",
    img,
    data_sample=new_result,
    draw_gt=False,
    wait_time=0,
    out_file=None,
    pred_score_thr=0.5
)
visualizer_now.show()